# Rapport sur les PVVIH de MUSO

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from dateutil.parser import parse
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import os
from dotenv import load_dotenv
import plotly.express as px
import openpyxl
import pymysql
from sqlalchemy import create_engine

## Connect to commcare

In [2]:
#Connecting to Commcare
load_dotenv('id_cc.env')
email = os.getenv('COMCARE_EMAIL')
password_cc = os.getenv('COMCARE_PASSWORD')

#Defining the driver
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(1000)

#Creating login function
def muso_beneficiaries():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/91fe1e2fc809dfd56a29bfce97cc9d15/'
    )
    #driver.find_element_by_xpath('//*[@id="id_auth-username"]').send_keys(email)
    driver.find_element(By.XPATH,'//*[@id="id_auth-username"]').send_keys(email)
    #driver.find_element_by_xpath('//*[@id="id_auth-password"]').send_keys(password_cc)
    driver.find_element(By.XPATH,'//*[@id="id_auth-password"]').send_keys(password_cc)
    driver.find_element(By.CSS_SELECTOR,'button[type=submit]').click()

#Muso beneficiaries
muso_beneficiaries()

#Download the database "All gardens"
#driver.find_element_by_xpath('//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element(By.XPATH,"//*[@id='download-export-form']/form/div[2]/div/div[2]/div[1]/button/span[1]").click()
#driver.find_element_by_xpath('//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()    
driver.find_element(By.XPATH,"//*[@id='download-progress']/div/div/div[2]/div[1]/form/a/span[1]").click()    

#-----------------------------------------------------------------------------------------------------------------------

#Household count
def muso_groups():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/91fe1e2fc809dfd56a29bfce973b4c0d/'
    )
    
muso_groups()

#Download the database "Household count"
#driver.find_element_by_xpath('//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element(By.XPATH,'//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
#driver.find_element_by_xpath('//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()
driver.find_element(By.XPATH,'//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()

True

## Import the data

In [3]:
muso_benef = pd.read_excel(f"~/Downloads/muso_beneficiaries {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", sheet_name = 'Cases')
muso_group = pd.read_excel(f"~/Downloads/muso_groupes {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx")

In [4]:
muso_group.rename(columns = {'is_graduated': 'graduated_group',
                            'is_inactive': 'inactive_group',
                            'closed': 'closed_group',
                            'caseid': 'caseid_group',
                            'office': 'office_group'}, inplace = True)

muso_benef.rename(columns = {'indices.muso_groupes': 'caseid_group'}, inplace = True)

group_info = muso_group[['caseid_group', 'office_name', 'office_group', 'graduated_group', 'inactive_group', 'closed_group']]

## Concat to check good codes and to add info status to infos beneficiaries

In [5]:
muso_sheet = muso_benef.merge(group_info, on = 'caseid_group', how = 'left')
muso_sheet.shape

(52570, 81)

In [6]:
#connecting to HIV Haiti to get all codes on HIV Haiti
load_dotenv('.env')
user = os.getenv('USRCaris')
password_hiv = os.getenv('PASSCaris')
host = os.getenv('HOSTCaris')
db = os.getenv('DBCaris')

conn_text = f'mysql+pymysql://{user}:{password_hiv}@{host}/{db}'
engine = create_engine(conn_text)

sql2 = 'select * from patient'
all_patient = pd.read_sql_query(sql2, engine, parse_dates = True)
all_patient.rename(columns = {'patient_code': 'patient_code_pv'}, inplace = True)

True

In [7]:
all_patient['check_patient_code'] = all_patient['patient_code_pv']

all_patient.columns

Index(['id', 'city_code', 'hospital_code', 'patient_number', 'patient_code_pv',
       'linked_to_id_patient', 'which_program', 'muso_case_id', 'created_at',
       'updated_at', 'created_by', 'updated_by', 'check_patient_code'],
      dtype='object')

In [8]:
# hhm_club = hhm_club.merge(OVC[['patient_code', 'Commune']], on = 'patient_code', how = 'left')

muso_sheet = muso_sheet.merge(all_patient[['patient_code_pv', 'check_patient_code']], on = 'patient_code_pv', how = 'left')

#merging is_pvvih and is_pvhiv
muso_sheet['is_pvvih'].fillna("---", inplace = True)
muso_sheet['is_pvvih'] = muso_sheet['is_pvhiv'].where(muso_sheet['is_pvvih'].isin(["---"]), 
                                           other=muso_sheet['is_pvvih'])

In [9]:
muso_sheet = muso_sheet[muso_sheet.group_name != 'test Shisu']

## Active benef and PVVIH

In [10]:
#filtering

active_benef = muso_sheet[(muso_sheet['why_remove_the_member'] == '---') & 
                          (muso_sheet['graduated_group'] != '1') &
                         (muso_sheet['inactive_group'] != '1') & 
                         (muso_sheet['closed_group'] != True)]

pvvih = active_benef[(active_benef['is_pvvih'] == '1')]

In [11]:
muso_sheet.to_excel('Muso_data.xlsx', index = False)

## Infos global

In [12]:
print(f"Informations clés: Membres actifs : {active_benef.shape[0]} , PVVIH: {pvvih.shape[0]}.")

Informations clés: Membres actifs : 26515 , PVVIH: 2939.


## Patient PV avec code

In [13]:
patient_code = pvvih[~(pvvih['patient_code_pv'].isin(['---', '//']))]
patient_code.shape[0]

2873

In [14]:
patient_code.to_excel(f"MUSO_PV_w_code {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", index = False)

## PV sans code



In [15]:
pv = pvvih[pvvih['patient_code_pv'].isin(['---', '//'])]
pv_wc = pv[['caseid', 'first_name', 'last_name', 'dob', 'gender', 'office_name', 'office_group','is_pvvih', 'patient_code_pv',
            'check_patient_code', 'group_name','group_code', 'group_commune', 'group_departement', 'phone', 'inactive_date', 'is_abandoned',
                   'officer_name', 'muso_start_date']]

pv_wc['office_name'] = np.where(pv_wc['office_name'] == '---', pv_wc['office_group'], pv_wc['office_name'])
pv_wc.drop(['office_group'], axis = 1, inplace = True)
pv_wc['gender']=pd.to_numeric(pv_wc['gender'],errors="coerce")
pv_wc['gender']=pv_wc['gender'].replace([1,2],["M","F"])
pv_wc.head()

,caseid,first_name,last_name,dob,gender,office_name,is_pvvih,patient_code_pv,check_patient_code,group_name,group_code,group_commune,group_departement,phone,inactive_date,is_abandoned,officer_name,muso_start_date
13561,732dce8ee80a4428961b7fc9ee9fb943,Rosemica,Dor,1994-01-01,F,GON,1,---,NaN,L'union fait la force,196,Gonaïves,Artibonite,NaN,NaN,0,---,---
16988,b13659e8b17b464da0642e9e8a187f05,Rose Sendy,Vilson,1991-10-28,F,GON,1,---,NaN,Tet Ansanm poun Vanse,237,Gonaïves,Artibonite,NaN,NaN,0,---,---
21414,2183f98c1dbe4364907b7d5609bbfd9c,Chrisnaelle,Vertus,1984-01-01,F,GON,1,---,NaN,Sere pou Chofe,307,Petite Rivière de l'Artibonite,Artibonite,NaN,NaN,0,---,2020-08-25
32775,114255e3-2ae5-438f-b420-037c6033654e,Alourdes,Clermont,1985-09-10,F,GON,1,---,NaN,LAVI MIYÒ,---,---,---,NaN,---,---,---,2019-11-11
33340,7900465b-7646-42b8-b1d6-e3aa4abf890a,Mysthana,Charles,1984-08-04,F,GON,1,---,NaN,LAVI MIYÒ,---,---,---,NaN,---,---,---,2019-11-11


In [16]:
pv_wc.to_excel(f"MUSO_PVVIH_sans_code {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", index = False)

In [17]:
pv_wc.shape[0]

66

## Export final file

In [18]:
df = pv_wc
column_name = 'office_name'
unique_values = df[column_name].unique()

with pd.ExcelWriter(f"./By_office/PV_sans_code {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", engine = 'openpyxl') as writer:
    for unique_value in unique_values:
        df_output = df[df[column_name].str.contains(unique_value)]
        df_output.to_excel(writer, sheet_name = unique_value, index = False)

In [19]:
patient_code = patient_code[['caseid', 'first_name', 'last_name', 'dob', 'gender', 'office_name', 'office_group','is_pvvih', 'patient_code_pv',
            'check_patient_code', 'group_name','group_code', 'group_commune', 'group_departement', 'phone', 'inactive_date', 'is_abandoned',
                   'officer_name', 'muso_start_date']]
patient_code['office_name'] = np.where(patient_code['office_name'] == '---', patient_code['office_group'], patient_code['office_name'])
patient_code.drop(['office_group'], axis = 1, inplace = True)
patient_code['gender']=pd.to_numeric(patient_code['gender'],errors="coerce")
patient_code['gender']=patient_code['gender'].replace([1,2],["M","F"])
df1 = patient_code
column_name = 'office_name'
unique_values = df1[column_name].unique()

with pd.ExcelWriter(f"./By_office/PV_checking_code {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", engine = 'openpyxl') as writer:
    for unique_value in unique_values:
        df1_output = df1[df1[column_name].str.contains(unique_value)]
        df1_output.to_excel(writer, sheet_name = unique_value, index = False)

In [ ]:
driver.quit()